# 根据 txt 以及 json一一对应的数据，生成

In [ ]:
import os
import json

# 定义文件目录
directory = os.path.join(os.getcwd(), "prompt_txt_json")# lasson

# 获取目录下所有 .txt 和 .json 文件
txt_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
json_files = [f for f in os.listdir(directory) if f.endswith('.json')]

cnt = 0
result_list = []
# 遍历所有 .txt 文件
for txt_file in txt_files:
    # 找到对应的 .json 文件
    json_file = txt_file.replace('.txt', '_result.json')
    
    if json_file in json_files:
        # 读取 .txt 文件内容
        with open(os.path.join(directory, txt_file), 'r') as file:
            instruction = file.read().strip()

        # 读取 .json 文件内容
        with open(os.path.join(directory, json_file), 'r') as file:
            json_content = json.load(file)
            output = json.dumps(json_content, ensure_ascii=False)
        cnt += 1
        # 提取文件名前缀作为name
        name = txt_file.split('_')[0]
        if "other" in name:
            name = "other_disease"
        elif "pseudo" in name:
            name = "pseudo_papilledema"
        elif "Synonms" in name:
            name = "real_papilledema"
        name = "generate_" + name + "_referral_letter"

        # 创建目标格式的字典
        result = {
            "id": f"seed_task_{cnt}",
            "name": name,
            "instruction": instruction,
            "instances": [{"input": "", "output": output}],
            "is_classification": False
        }
        result_list.append(result)

with open('seed_tasks.jsonl', 'w', encoding='utf-8') as output_file:
    for result in result_list:
        output_file.write(json.dumps(result, ensure_ascii=False) + '\n')

In [9]:
print(result_list)

[{'id': 'seed_task_1', 'name': 'other', 'instruction': 'Role: You are an experienced doctor with knowledge of electronic medical records related to many diseases.\n\nInstruction: Please generate a referral letter for a non-ophthalmic disease. The letter should be within 150 words and include a clear referral reason along with some basic patient information. The referral reason should be in a separate paragraph dedicated entirely to it. Do not use a table for patient information. If a table is used, it should be placed in the middle of the letter. Use different beginnings and endings for different letters. The beginning and ending of the letter should be brief, with minimal email and address details, and include simulated information.\n\nOutput your result in the format: {"is_referral_letter": boolean, "is_Papilledema": boolean, "whole_letter": "", "referral_content": ""}\n\nRule For is_Papilledema: If the referral letter contains any words from the provided Synonyms, then is_Papilledem

# 使用 mak的98条数据，生成prompt，放入seed_tasks.jsonl文件里

In [15]:
def make_single_instruction_using_mak(txt):
    instruction = f"""Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.

    Instruction : please generate one referral letter according to the medical records separeted by ###. It need to be within 150 words.And it should include the very clear referral reason and few simulated patient basic information. The referral reason should be in a whole separate paragraph, which means this paragraph shoule be all about the referral reason. Also the referral reason should be short and precise.You don't use table to contain patient information. If use table, This table should be in the middle part of the letter. Just use different begining and ending information in different letters.The letter begining as well as ending should be short, which means the email and address can be as short as possible. And the it should have simulated information. 

    ###
    {txt}
    ###

    output your result in format: {{"is_referral_letter": boolean, "is_Papilledema": boolean, "whole_letter": "", "referral_content": "",}}

    Rule For is_Papilledema : false.

    Rule For is_referral_letter : if you are producing a referral letter, then it is true.

    Rule For whole_letter : this is the content you generate.

    Rule For referral_content : this content should be a whole paragraph which tells Patient need referral. If the referral_letter contains this content, you should include it. If the letter doesn't contain related information, then it should be null.
    """
    return instruction


In [16]:
import pandas as pd
import json

# 加载 CSV 文件
file_path = './prompt_txt_json/filtered_data_unique_8_copy.csv'
data = pd.read_csv(file_path)

# 构造基于 'patient' 列的 JSON 数据
cnt = 13 # 因为从13开始
result_list = []

# 遍历每一行并构造 JSON 结构
for index, row in data.iterrows():    
    instruction = make_single_instruction_using_mak(row['patient'].strip())
    output = json.dumps({
            "is_referral_letter": True,
            "is_Papilledema": False, # 只有索引为4， seed_task_16的数据，此处为true。看gpt查出来的，所以手动改了数据，而没写代码
            "whole_letter": "nothing",
            "referral_content": "nothing"
    }, ensure_ascii=False)
    result = {
        "id": f"seed_task_{cnt}",
        "name": "generate_mak_other_disease_referral_letter",
        "instruction": instruction,
        "instances": [{"input": "", "output": output}],
        "is_classification": False
    }
    
    result_list.append(result)
    cnt += 1

In [17]:
# print(len(result_list))
# print(result_list[0])

{'id': 'seed_task_13', 'name': 'generate_mak_other_disease_referral_letter', 'instruction': 'Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.\n\n    Instruction : please generate one referral letter according to the medical records separeted by ###. It need to be within 150 words.And it should include the very clear referral reason and few simulated patient basic information. The referral reason should be in a whole separate paragraph, which means this paragraph shoule be all about the referral reason. Also the referral reason should be short and precise.You don\'t use table to contain patient information. If use table, This table should be in the middle part of the letter. Just use different begining and ending information in different letters.The letter begining as well as ending should be short, which means the email and address can be as short as possible. And the it should have simulated information. \n\n    ###\n    Case 

In [18]:
# 将 JSON 数据保存到文件
with open('seed_tasks.jsonl', 'a', encoding='utf-8') as output_file:
    for result in result_list:
        output_file.write(json.dumps(result, ensure_ascii=False) + '\n')

# 重新读取 seed_tasks.jsonl 看看每条数据的格式是否正确

In [19]:
import json
data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

In [21]:
print(len(data))

110


## 读取第一条数据，查看形式

In [19]:
sample = data[0]
output = sample["instances"][0]["output"]
print(type(output))
print(output)

<class 'str'>
{"is_referral_letter": true, "is_Papilledema": false, "whole_letter": "Dr. Sarah Thompson\nABC Medical Clinic\n789 Oak St, Suite 204\nTown, State, 34567\nOctober 5, 2024\n\nDr. John Miller\nNeurology Specialists\n123 Pine Rd, Suite 101\nTown, State, 67890\n\nDear Dr. Miller,\n\nI am referring Ms. Jane Doe, a 60-year-old female, for urgent evaluation due to suspected multiple sclerosis. Ms. Doe has been experiencing numbness, tingling, and muscle weakness over the past month.\n\n**Referral Reason:**\nBased on the patient's symptoms and initial examination findings, there is a high suspicion of multiple sclerosis. An MRI showed multiple demyelinating lesions in the brain. Given her family history of neurological disorders, further neurological assessment is necessary.\n\nYour expertise is needed to confirm the diagnosis and develop an appropriate treatment plan to manage her condition.\n\nBest regards,\nDr. Sarah Thompson\nABC Medical Clinic", "referral_content": "Based on 

In [20]:
output_dict = json.loads(output)
print(type(output_dict))

<class 'dict'>


In [22]:
print(output_dict["is_referral_letter"])
print(output_dict["is_Papilledema"])

print(output_dict["referral_content"])

True
False
Based on the patient's symptoms and initial examination findings, there is a high suspicion of multiple sclerosis. An MRI showed multiple demyelinating lesions in the brain. Given her family history of neurological disorders, further neurological assessment is necessary.


## 读取第16条数据，查看形式

In [25]:
sample_last = data[16]
output = sample_last["instances"][0]["output"]
print(type(output))
print(output)

output_dict = json.loads(output)
print(type(output_dict))
print(output_dict["is_referral_letter"])
print(output_dict["is_Papilledema"])

print(output_dict["referral_content"])

<class 'str'>
{"is_referral_letter": true, "is_Papilledema": false, "whole_letter": "nothing", "referral_content": "nothing"}
<class 'dict'>
True
False
nothing


# 读取 ./lasson/regen.json

## 清理每条regen里边的数据，并且加上 seed_tasks里边对应的信息

In [ ]:
import json
seed_tasks_data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        seed_tasks_data.append(json.loads(line.strip()))

In [4]:
import json
with open('./lasson/regen.json', 'r', encoding='utf-8') as f:
    regen_data = json.loads(f.read())

In [5]:
print(len(regen_data))

110


In [41]:
import json
seed_tasks_data = []
with open('seed_tasks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        seed_tasks_data.append(json.loads(line.strip()))

with open('./lasson/regen.json', 'r', encoding='utf-8') as f:
    regen_data = json.loads(f.read())
        
# 用于存储成功处理的数据
updated_regen_data = []

for i, regen_sample in enumerate(regen_data):
    regen_output = regen_sample["output"]
    
    if re.findall(r"^```json\n|```\n$", regen_output):
        
        # 清理开头和结尾的 ```json 和 ```\n
        regen_output = re.sub(r'^```json\n', '', regen_output, flags=re.MULTILINE)
        regen_output = re.sub(r"\n```$", '', regen_output, flags=re.MULTILINE)
        
        # 清理四个字段后边的 \n
        regen_output = re.sub(r',\n\s+"is_Papilledema"', ', "is_Papilledema"', regen_output)
        regen_output = re.sub(r',\n\s+"whole_letter"', ', "whole_letter"', regen_output)
        regen_output = re.sub(r',\n\s+"referral_content"', ', "referral_content"', regen_output)
        regen_output = re.sub(r'\n}', '}', regen_output)
        

#         print(type(regen_output))
#         print(regen_output)
    try:
        regen_output_json = json.loads(regen_output)
        a = regen_output_json["is_Papilledema"]
        b = regen_output_json["is_referral_letter"]
        c = regen_output_json["whole_letter"]
        d = regen_output_json["referral_content"]
        
         # 添加 instruction 字段
        regen_sample["gpt_generate_data_instruction"] = seed_tasks_data[i]["instruction"]
        
        # 添加 id 字段
        regen_sample["id"] = seed_tasks_data[i]["id"]
        
        # 将更新后的 JSON 对象转换回字符串并存储回 regen_sample 中
        regen_sample["output"] = json.dumps(regen_output_json, ensure_ascii=False, indent=4)
        updated_regen_data.append(regen_sample)

    except json.JSONDecodeError:
        print(i + 1)
#         print(regen_output)
        print()

# 将更新后的 regen_data 写回到一个新的文件
with open('./lasson/updated_regen.json', 'w', encoding='utf-8') as f:
    json.dump(updated_regen_data, f, ensure_ascii=False, indent=4)

19

31

39

48

91

94

98



In [42]:
print(len(updated_regen_data))

103


## 创建 CSV 文件

In [45]:
import pandas as pd
# 创建 CSV 文件
csv_data = []

for item in updated_regen_data:
    output_json = json.loads(item["output"])
    csv_data.append({
        "id": item["id"],
        "name": item["name"],
        "gpt_generate_data_instruction": item["gpt_generate_data_instruction"],
        "is_Papilledema": output_json["is_Papilledema"],
        "whole_letter": output_json["whole_letter"],
        "referral_content": output_json["referral_content"]
    })

# 创建 DataFrame
df = pd.DataFrame(csv_data)

# 保存为 CSV 文件
csv_file_path = './lasson/updated_regen.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f"CSV file has been saved to {csv_file_path}")

CSV file has been saved to ./lasson/updated_regen.csv


In [3]:
df.head(2)

,id,name,gpt_generate_data_instruction,is_Papilledema,whole_letter,referral_content
0,seed_task_1,generate_other_disease_referral_letter,Role: You are an experienced doctor with knowl...,False,Dr. Jane Smith\nSunrise Medical Center\n123 He...,Mr. Doe has recently exhibited symptoms includ...
1,seed_task_2,generate_other_disease_referral_letter,Role: You are an experienced doctor familiar w...,False,Dr. Amy Johnson\nHealthy Living Medical Center...,Mr. Smith has been experiencing persistent joi...


In [22]:
# regen_sample = regen_data[10] # 3有json这个字符， 10 前后有\n
# regen_output = regen_sample["output"]
# print(type(regen_output))
# print(regen_output)

# regen_output_json = json.loads(regen_output)

# print(regen_output_json["is_referral_letter"])
# print(regen_output_json["is_Papilledema"])
# print(regen_output_json["whole_letter"])
# print(regen_output_json["referral_content"])
# print()

# 将数据处理成 llama2可以直接使用的形式

## 添加instruction以及分割数据集

In [1]:
import pandas as pd

df = pd.read_csv("./lasson/updated_regen.csv")

In [2]:
df.head()

,id,name,gpt_generate_data_instruction,is_Papilledema,whole_letter,referral_content
0,seed_task_1,generate_other_disease_referral_letter,Role: You are an experienced doctor with knowl...,False,Dr. Jane Smith\nSunrise Medical Center\n123 He...,Mr. Doe has recently exhibited symptoms includ...
1,seed_task_2,generate_other_disease_referral_letter,Role: You are an experienced doctor familiar w...,False,Dr. Amy Johnson\nHealthy Living Medical Center...,Mr. Smith has been experiencing persistent joi...
2,seed_task_3,generate_other_disease_referral_letter,Role: You are an experienced doctor with acces...,False,"Dr. Samantha Greene, \nMediCare Clinic\nsamant...",Mr. Doe has been experiencing persistent hyper...
3,seed_task_4,generate_other_disease_referral_letter,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...",False,"Dr. Smith,\n\nI hope this message finds you we...",The patient is being referred to you for furth...
4,seed_task_5,generate_pseudo_papilledema_referral_letter,"Synonyms: [""Papilledema"", ""Swollen discs"", ""In...",False,"To: Dr. Sarah Thompson, Optometry Clinic\nemai...",Mr. John Doe presents with signs that could be...


In [3]:
txt = """Role : You are a experienced doctor who have memory of electronic medical records related to many diseases.

Instruction : please extract the referral reason from the following referral letter  separeted by ###.

Rule For referral reason: this content should be a whole paragraph which tells Patient need referral. If the referral_letter contains this content, you should include it. If the letter doesn't contain related information, then it reply no referral reason

output your result directly"""

df["instruction"] = txt

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=2024)

In [6]:
train_data.head()

,id,name,gpt_generate_data_instruction,is_Papilledema,whole_letter,referral_content,instruction
98,seed_task_106,generate_mak_other_disease_referral_letter,Role : You are a experienced doctor who have m...,False,"Dear Dr. John Doe,\n\nI am referring a four-ye...","Given the complexity of Jimmy's condition, he ...",Role : You are a experienced doctor who have m...
10,seed_task_11,generate_real_papilledema_referral_letter,"Synonyms: [""Papilledema"", ""Swollen discs"", ""In...",True,Dr. Anthony Smith\nOphthalmology Department\nS...,"During a routine eye examination, I observed s...",Role : You are a experienced doctor who have m...
24,seed_task_26,generate_mak_other_disease_referral_letter,Role : You are a experienced doctor who have m...,False,"Dear Dr. Brown,\n\nI am writing to refer Emily...",The referral is due to Emily's complex partial...,Role : You are a experienced doctor who have m...
3,seed_task_4,generate_other_disease_referral_letter,"Synonyms: [""Papilledema"",""Swollen discs"",""Indi...",False,"Dr. Smith,\n\nI hope this message finds you we...",The patient is being referred to you for furth...,Role : You are a experienced doctor who have m...
63,seed_task_68,generate_mak_other_disease_referral_letter,Role : You are a experienced doctor who have m...,False,"Dear Dr. Jones,\n\nI hope this letter finds yo...",The primary reason for this referral is the id...,Role : You are a experienced doctor who have m...


In [7]:
df.columns

Index(['id', 'name', 'gpt_generate_data_instruction', 'is_Papilledema',
       'whole_letter', 'referral_content', 'instruction'],
      dtype='object')

## 保存train

In [8]:
import csv
import json

# 定义输入 CSV 文件和输出 JSONL 文件的路径
output_jsonl_file = '../lasson_LLM/train.jsonl'


# 读取 CSV 文件并写入 JSONL 文件
with open(output_jsonl_file, mode='w', encoding='utf-8') as train_jsonl_file:
    for index, row in train_data.iterrows():
        train_jsonl_file.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print(f"CSV data has been converted to JSONL format and saved to {output_jsonl_file}.")


CSV data has been converted to JSONL format and saved to ../lasson_LLM/train.jsonl.


## 保存 test

In [9]:
# 定义输入 CSV 文件和输出 JSONL 文件的路径
output_jsonl_file = '../lasson_LLM/test.jsonl'

# 读取 CSV 文件并写入 JSONL 文件
with open(output_jsonl_file, mode='w', encoding='utf-8') as test_jsonl_file:
    for index, row in test_data.iterrows():
        test_jsonl_file.write(json.dumps(row.to_dict(), ensure_ascii=False) + '\n')

print(f"CSV data has been converted to JSONL format and saved to {output_jsonl_file}.")

CSV data has been converted to JSONL format and saved to ../lasson_LLM/test.jsonl.
